In [ ]:
%load_ext autoreload
%autoreload 2
%run notebook_setup.py

In [ ]:
import sys
import os
from backtest import *
from pareto_store import *
from pure_funcs import calc_drawdowns


In [ ]:
config = load_config('configs/template.json')
config['live']['approved_coins'] = 'BTC'
config['backtest']['exchanges'] = ['bybit', 'binance']
config['backtest']['combine_ohlcvs'] = True
config['backtest']['start_date'] = '2023-01-01'
config['live']['minimum_coin_age_days'] = 30
config = format_config(config)
res = await prepare_hlcvs_mss(config, 'combined' if config['backtest']['combine_ohlcvs'] else config['backtest']['exchanges'][0])
coins, hlcvs, mss, results_path, cache_dir, btc_usd_prices = res
#prices = btc_usd_prices
hlcv = hlcvs[:,0]

In [ ]:
plt.plot(hlcv[:,2])

In [ ]:
results = {(0.05172, 1.346, 115.8, 0.03807, 0.02142, 0.01076, 0.003991): None}
eligible = {}

In [ ]:
len(eligible)

In [ ]:
def perform_backtest(
    hlcv,
    key,
):
    fee_rate = 0.00055
    fills, equities = pbr.backtest_trailing_flip(
        hlcv,
        *key,
        fee_rate,
    )
    fdf = pd.DataFrame(fills, columns=["index", "pnl", "fee_paid", "balance", "qty", "price", "psize", "pprice", "upnl", "type"]).set_index('index')
    edf = pd.Series(equities)
    fdf.loc[:,'wallet_exposure'] = fdf.psize.abs() * fdf.pprice / fdf.balance
    fdf.loc[:,'equity'] = fdf.balance + fdf.upnl
    drawdowns = calc_drawdowns(edf)
    drawdown_worst = abs(drawdowns.min())
    gain = edf.iloc[-1] / edf.iloc[0]
    n_days = len(edf) / 60 / 24
    adg = gain ** (1 / n_days) - 1 if gain > 0.0 else gain
    return {'drawdown_worst': drawdown_worst, 'adg': adg}

In [ ]:
begin_harmony_search = 20
harmony_memory_size = 20
limit_drawdown_worst = 0.5
n_iters = 40000

for k in results:
    if results[k] is None:
        results[k] = perform_backtest(hlcv, k)
        print('backtested', k, results[k])
        
eligible = {k: v for k, v in results.items() if v and v['drawdown_worst'] < limit_drawdown_worst}
for z in range(n_iters):
    if len(eligible) > begin_harmony_search:
        mode = 'harmony_search'
        resfs = sorted(eligible.items(), key=lambda x: x[1]['adg'])[-harmony_memory_size:]
        #if len(resfs) >= harmony_memory_size:
        #    eligible = {x[0]: eligible[x[0]] for x in resfs}
        candidate = []
        for i in range(len(resfs[0][0])):
            vals = [resfs[k][0][i] for k in range(len(resfs))]
            candidate.append((np.random.choice(vals) + np.random.uniform(-0.000001, 0.000001)) * np.random.uniform(0.9, 1.1))
        initial_qty_pct, double_down_factor, wallet_exposure_limit, trailing_threshold_pct_profit, trailing_threshold_pct_loss, trailing_retracement_pct_profit, trailing_retracement_pct_loss = candidate
    else:
        mode = 'random'
        initial_qty_pct = np.random.uniform(0.01, 1.0)
        double_down_factor = np.random.uniform(1.1, 3.0)
        wallet_exposure_limit = np.random.uniform(10.0, 100.0)
        trailing_threshold_pct_profit = np.random.uniform(0.01, 0.03)
        trailing_threshold_pct_loss = np.random.uniform(0.01, 0.03)
        trailing_retracement_pct_profit = np.random.uniform(0.00001, 0.01)
        trailing_retracement_pct_loss = np.random.uniform(0.00001, 0.01)
        
    key = (
        pbr.round_dynamic(initial_qty_pct, 4),
        pbr.round_dynamic(double_down_factor, 4),
        pbr.round_dynamic(wallet_exposure_limit, 4),
        pbr.round_dynamic(trailing_threshold_pct_profit, 4),
        pbr.round_dynamic(trailing_threshold_pct_loss, 4),
        pbr.round_dynamic(trailing_retracement_pct_profit, 4),
        pbr.round_dynamic(trailing_retracement_pct_loss, 4),
    )
    if key in results:
        continue
    result = perform_backtest(hlcv, key)
    results[key] = result
    if result['drawdown_worst'] < limit_drawdown_worst:
        eligible[key] = result
        print(z, len(eligible), mode, key, eligible[key])
        

In [ ]:
resf = {k: v for k, v in results.items() if v['drawdown_worst'] < limit_drawdown_worst}
resfs = sorted(resf.items(), key=lambda x: x[1]['adg'])[-harmony_memory_size:]
resfs

In [ ]:
if not resfs:
    print("No eligible candidates found.")
    # Skip further processing
    raise ValueError("No eligible candidates found. Adjust parameters or increase iterations.")

candidate = resfs[-1][0]
fee_rate = 0.00055
initial_qty_pct, double_down_factor, wallet_exposure_limit, trailing_threshold_pct_profit, trailing_threshold_pct_loss, trailing_retracement_pct_profit, trailing_retracement_pct_loss = candidate
fills, equities = pbr.backtest_trailing_flip(
    hlcv,
    *candidate,
    fee_rate,
)
fdf = pd.DataFrame(fills, columns=["index", "pnl", "fee_paid", "balance", "qty", "price", "psize", "pprice", "upnl", "type"]).set_index('index')
edf = pd.Series(equities)
fdf.loc[:,'wallet_exposure'] = fdf.psize.abs() * fdf.pprice / fdf.balance
fdf.loc[:,'equity'] = fdf.balance + fdf.upnl
drawdowns = calc_drawdowns(edf)
drawdown_worst = abs(drawdowns.min())
gain = edf.iloc[-1] / edf.iloc[0]
n_days = len(edf) / 60 / 24
adg = gain ** (1 / n_days) - 1
print('drawdown_worst', drawdown_worst)
print('n_days', n_days)
print('n fills', len(fdf))
print('adg', adg)
print('WE max', fdf.wallet_exposure.max())

In [ ]:
edf.plot(logy=True)

In [ ]:
edf.plot(logy=False)

In [ ]:
cost_pct = (fdf.qty.abs() * fdf.price) / fdf.balance
cost_pct.groupby(cost_pct.index // 1440).sum().mean()

In [ ]:
fdf.iloc[:60]

In [ ]:
# Initialize variables to track flips and streaks
current_flips = 0
flip_streaks = []  # List to store the lengths of all negative PnL streaks
max_flips = 0      # Variable to track the maximum flips

# Iterate through the 'pnl' column in the fills DataFrame
for pnl in fdf['pnl']:
    if pnl <= 0:
        # Increment the current streak of negative PnL
        current_flips += 1
    else:
        if current_flips > 0:
            # Add the current streak to the list of streaks
            flip_streaks.append(current_flips)
            # Update the maximum flips if the current streak is the longest
            max_flips = max(max_flips, current_flips)
        # Reset the current streak
        current_flips = 0

# After the loop, ensure the last streak is considered
if current_flips > 0:
    flip_streaks.append(current_flips)
    max_flips = max(max_flips, current_flips)

# Calculate the average number of flips
if flip_streaks:
    avg_flips = sum(flip_streaks) / len(flip_streaks)
    print(f"maximum flips: {max_flips}")
    print(f"average flips: {avg_flips:.2f}")
else:
    print("No negative PnL streaks found.")

In [ ]:
fdf.sort_values('wallet_exposure')

In [ ]:
i = 607126
n = 7000
fdfc = fdf.loc[i-n:i+n]
buys = fdfc[fdfc.qty > 0.0]
sells = fdfc[fdfc.qty < 0.0]
pd.DataFrame(hlcv[:,2]).loc[i-n:i+n].plot()
buys.price.plot(style='bo')
sells.price.plot(style='ro')

In [ ]:
fdfc

In [ ]:
edf.loc[fdfc.index[0]:fdfc.index[-1]].plot()

In [ ]:
fdf.sort_values('wallet_exposure').iloc[-10:]